In [1]:
import pandas as pd
import os,sys
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from collections import Counter
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression 
from sklearn.pipeline import Pipeline
from joblib import dump, load


In [8]:
doclist = pd.read_excel('data/merged.xlsx','cleaned')
doclist = doclist.reset_index()

In [9]:
doclist.shape

(6533, 7)

In [10]:
doclist['CREDIBLE'].value_counts()

-1    4469
 1    1948
 0     116
Name: CREDIBLE, dtype: int64

In [ ]:
DOCS_DIR = '/media/ludwig/story/DecisionRunDocs/docs_28topics/'
SAVE_DIR = 'model/'
docs = []
for docname in doclist['DOCID']:
    with open(DOCS_DIR + docname) as fh:
        docs.append(fh.read())

In [ ]:
doclist['DOC'] = docs
doclist = doclist[~doclist['DOCID'].duplicated()]
doclist

In [13]:
doclist['CREDIBLE'].value_counts()

-1    4469
 1    1948
 0     116
Name: CREDIBLE, dtype: int64

In [14]:
train, test = train_test_split(doclist,test_size = .2, random_state = 156)

In [15]:
doclist.shape

(6533, 7)

In [13]:
# test, val   = train_test_split(test,test_size = .5, random_state = 156)
# del train['DOC']
# del test['DOC']

In [14]:

X_train = train['DOC'].values.tolist()
X_test  = test['DOC'].values.tolist()
# X_val   = val['DOC'].values.tolist()

y_train, y_test = train['CREDIBLE'], test['CREDIBLE']

In [10]:
from sklearn.linear_model import LogisticRegression 
from sklearn.pipeline import Pipeline
from joblib import dump, load

## Input to sklearn
vect = CountVectorizer(ngram_range=(4,4), analyzer='char', binary=True)
logreg = LogisticRegression()
pline  = Pipeline([('vectorizer', vect), ('logreg', logreg)])
pline.fit(X_train, y_train)


Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='char', binary=True, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(4, 4), preprocessor=None, stop_words=None,
        ...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [11]:
y_preds = pline.predict(X_test)

In [12]:
print('Accuracy (Test):', metrics.accuracy_score(y_test, y_preds))
# print('Test F1 score (Test):', metrics.f1_score(y_test, y_pred_class, average='macro'))
metrics.confusion_matrix(y_test, y_preds)

Accuracy (Test): 0.9462809917355371


array([[482,   1,  10],
       [  8,   1,   7],
       [ 11,   2, 204]])

## After some adjustments in the Data (e.g. correcting different jugments for same documents)

In [7]:
doclist = pd.read_excel('merged_v2.xlsx')
doclist = doclist.reset_index()
doclist = doclist[['DOCID','source','CREDIBLE.1']]
doclist.columns = ['DOCID', 'SOURCE', 'CREDIBLE']
doclist.head()

# for docname in doclist['DOCID']:
DOCS_DIR = '/media/ludwig/story/DecisionRunDocs/docs_28topics/'
docs = []
for docname in doclist['DOCID']:
    with open(DOCS_DIR + docname) as fh:
        docs.append(fh.read())
        
doclist['DOC'] = docs



In [8]:
doclist[~doclist['DOCID'].duplicated()]['CREDIBLE'].value_counts()

-1    2452
 1    1081
 0      94
Name: CREDIBLE, dtype: int64

### # of assessments per topic after duplicate cleaning

In [8]:
doclist = doclist[~doclist['DOCID'].duplicated()]
doclist['SOURCE'].value_counts()

exercise_scoliosis_NC.csv                                  738
thermotherapy_osteoarthritis_C.csv                         438
antibiotics_sinusitis_NC.csv                               380
alprazolam_depression_NC.csv                               331
beta-blockers_stroke_NC.csv                                232
antibiotics_otitis_media_C.csv                             200
surgery_cataract_removal_NC.csv                            150
Spironolactone_high_blood_pressure_C.csv                   126
Hormone_therapy_cardiovascular_disease_menopause_NC.csv    117
antibiotics_laryngitis_NC.csv                              112
pancreatic_enzymes_pancreatitis_C.csv                      111
screening_breast_cancer_C.csv                              106
vaccine_hepatitis_B_NC.csv                                 101
exercise_diabetes_C.csv                                     91
echinacea_common_cold_C.csv                                 89
exercise_neck_pain_C.csv                               

### Classification

In [19]:
train, test = train_test_split(doclist,test_size = .2, random_state = 156)
X_train = train['DOC'].values.tolist()
X_test  = test['DOC'].values.tolist()
y_train, y_test = train['CREDIBLE'], test['CREDIBLE']

In [20]:
## Input to sklearn
vect = CountVectorizer(ngram_range=(4,4), analyzer='char', binary=True)
logreg = LogisticRegression()
pline  = Pipeline([('vectorizer', vect), ('logreg', logreg)])
pline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='char', binary=True, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(4, 4), preprocessor=None, stop_words=None,
        ...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [21]:
y_pred_class = pline.predict(X_test)
print('Accuracy (Test):', metrics.accuracy_score(y_test, y_pred_class))
# print('Test F1 score (Test):', metrics.f1_score(y_test, y_pred_class, average='macro'))
metrics.confusion_matrix(y_test, y_pred_class)

Accuracy (Test): 0.9435261707988981


array([[478,   0,   8],
       [ 11,   3,  10],
       [ 10,   2, 204]])

## Kfold Validation

In [13]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=10, random_state=156, shuffle=True)
res = []

for train_index, test_index in kf.split(doclist):
#     print(train_index, test_index)
    X_train = doclist.iloc[train_index,:]['DOC'].values.tolist()
    X_test  = doclist.iloc[test_index,:]['DOC'].values.tolist()
    y_train, y_test = doclist.iloc[train_index,:]['CREDIBLE'], doclist.iloc[test_index,:]['CREDIBLE']
#     print(len(X_train),len(X_test),len(y_train),len(y_test))
    ## Input to sklearn
    vect = CountVectorizer(ngram_range=(4,4), analyzer='char', binary=True)
    logreg = LogisticRegression()
    pline  = Pipeline([('vectorizer', vect), ('logreg', logreg)])
    pline.fit(X_train, y_train)
    y_pred_class = pline.predict(X_test)
    print('Accuracy (Test):', metrics.accuracy_score(y_test, y_pred_class))
    res.append(metrics.accuracy_score(y_test, y_pred_class))

/usr/lib/python3/dist-packages/sklearn/linear_model/_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy (Test): 0.928374655647383


/usr/lib/python3/dist-packages/sklearn/linear_model/_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy (Test): 0.9614325068870524


/usr/lib/python3/dist-packages/sklearn/linear_model/_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


...

Accuracy (Test): 0.914364640883978


/usr/lib/python3/dist-packages/sklearn/linear_model/_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy (Test): 0.9419889502762431


/usr/lib/python3/dist-packages/sklearn/linear_model/_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy (Test): 0.9447513812154696


In [14]:
res

[0.928374655647383,
 0.9614325068870524,
 0.9586776859504132,
 0.9476584022038568,
 0.9586776859504132,
 0.9559228650137741,
 0.9421487603305785,
 0.914364640883978,
 0.9419889502762431,
 0.9447513812154696]

In [16]:
sum(res)/len(res)

0.9453997534359162

### Kfold validation by topics

In [13]:
topics = sorted(doclist['SOURCE'].value_counts().keys())

In [14]:
topics_nonchange = ['antibiotics_bronchitis','exercise_scoliosis','garlic_for_common_cold', 'garlic_common_cold']

In [15]:
topics = [t.replace('_C.csv','') for t in topics]
topics = [t.replace('_NC.csv','') for t in topics]
topics = [t.replace('_NC2.csv','') for t in topics]

In [16]:
def populate(lst):
    c   = [l + '_C.csv' for l in lst]
    nc  = [l + '_NC.csv' for l in lst]
    nc2 = [l + '_NC2.csv' for l in lst]
    return(c + nc + nc2)

In [17]:
topics_change = list(set(topics) - set(topics_nonchange))

In [24]:
kf   = KFold(n_splits = 5, random_state = 156, shuffle = True)
splt = kf.split(topics_change)

In [25]:
accs = []

for train_index, test_index in splt:
    vect = CountVectorizer(ngram_range=(4,4), analyzer='char', binary=True)
    logreg = LogisticRegression()
    pline  = Pipeline([('vectorizer', vect), ('logreg', logreg)])
    test_topics  = [topics_change[t] for t in test_index]
    train_topics = [topics_change[t] for t in train_index]
    train_topics = train_topics + topics_nonchange
    train_topics = populate(train_topics)
    test_topics  = populate(test_topics)
    train, test  = doclist[~doclist['SOURCE'].isin(test_topics)], doclist[doclist['SOURCE'].isin(test_topics)]
    train        = train[~train['DOCID'].isin(test['DOCID'])]
    X_train      = train['DOC'].values.tolist()
    X_test       = test['DOC'].values.tolist()
    y_train, y_test = train['CREDIBLE'], test['CREDIBLE']
    vect = CountVectorizer(ngram_range=(4,4), analyzer='char', binary=True)
    logreg = LogisticRegression()
    pline  = Pipeline([('vectorizer', vect), ('logreg', logreg)])
    pline.fit(X_train, y_train)
    y_pred_class = pline.predict(X_test)
    print(metrics.accuracy_score(y_test, y_pred_class))
    accs.append(metrics.accuracy_score(y_test, y_pred_class))


0.6070143884892086
0.3349019607843137
0.8895663956639567
0.6518987341772152
0.9299610894941635


In [26]:
sum(accs)/5

0.6826685137217716

In [27]:
from sklearn.model_selection import KFold
kf   = KFold(n_splits = 10, random_state = 156, shuffle = True)
splt = kf.split(topics_change)

In [28]:
accs = []

for train_index, test_index in splt:
    vect = CountVectorizer(ngram_range=(4,4), analyzer='char', binary=True)
    logreg = LogisticRegression()
    pline  = Pipeline([('vectorizer', vect), ('logreg', logreg)])
    test_topics  = [topics_change[t] for t in test_index]
    train_topics = [topics_change[t] for t in train_index]
    train_topics = train_topics + topics_nonchange
    train_topics = populate(train_topics)
    test_topics  = populate(test_topics)
    train, test  = doclist[~doclist['SOURCE'].isin(test_topics)], doclist[doclist['SOURCE'].isin(test_topics)]
    train        = train[~train['DOCID'].isin(test['DOCID'])]
    X_train      = train['DOC'].values.tolist()
    X_test       = test['DOC'].values.tolist()
    y_train, y_test = train['CREDIBLE'], test['CREDIBLE']
    vect = CountVectorizer(ngram_range=(4,4), analyzer='char', binary=True)
    logreg = LogisticRegression()
    pline  = Pipeline([('vectorizer', vect), ('logreg', logreg)])
    pline.fit(X_train, y_train)
    y_pred_class = pline.predict(X_test)
    print(metrics.accuracy_score(y_test, y_pred_class))
    accs.append(metrics.accuracy_score(y_test, y_pred_class))


0.8581314878892734
0.5224787363304981
0.46348733233979134
0.19536423841059603
0.9904264577893821
0.45871559633027525
0.6837209302325581
0.8818181818181818
1.0
0.6732673267326733


In [29]:
sum(accs)/10

0.6727410287873229

### 10 Fold

In [185]:
kf   = KFold(n_splits = 10, random_state = 156, shuffle = True)
splt = kf.split(topics_change)

In [186]:
accs = []

for train_index, test_index in splt:
    test_topics  = [topics_change[t] for t in test_index]
    train_topics = [topics_change[t] for t in train_index]
    train_topics = train_topics + topics_nonchange
    train_topics = populate(train_topics)
    test_topics  = populate(test_topics)
    train, test  = doclist[~doclist['SOURCE'].isin(test_topics)], doclist[doclist['SOURCE'].isin(test_topics)]
    train        = train[~train['DOCID'].isin(test['DOCID'])]
    train        = train[~train['DOCID'].duplicated()]
    test         = test[~test['DOCID'].duplicated()]
    X_train      = train['DOC'].values.tolist()
    X_test       = test['DOC'].values.tolist()
    y_train, y_test = train['CREDIBLE'], test['CREDIBLE']
    vect = CountVectorizer(ngram_range=(4,4), analyzer='char', binary=True)
    logreg = LogisticRegression()
    pline  = Pipeline([('vectorizer', vect), ('logreg', logreg)])
    pline.fit(X_train, y_train)
    y_pred_class = pline.predict(X_test)
    print(metrics.accuracy_score(y_test, y_pred_class))
    accs.append(metrics.accuracy_score(y_test, y_pred_class))


0.21933085501858737
0.5198187995469988
0.8475750577367206
0.8995983935742972
0.5350877192982456
0.6196377502383222
0.9236111111111112
0.9188118811881189
0.5742574257425742
0.6666666666666666


In [188]:
sum(accs)/10

0.6724395660121643

In [190]:
[round(a,3) for a in accs]

[0.219, 0.52, 0.848, 0.9, 0.535, 0.62, 0.924, 0.919, 0.574, 0.667]